In [5]:
from basicTrainingData import y_test, X_test, tags, prey_train, preX_train, dict_vectorizer, add_basic_features, y_train, y_val, simple_train_sentences
from keras.models import load_model
import re
import pickle 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from os import system, listdir, path
import json
from nltk.corpus import wordnet
from collections import Counter
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


#Deep learning model
training_model = load_model('cnn.h5')
y_pred = training_model.model.predict(X_test, verbose=1).argmax(-1)
y_true = y_test.argmax(-1)

#Encoding model
pkl_file = open('Departure_encoder.pkl', 'rb')
encoder_model = pickle.load(pkl_file) 
pkl_file.close()



D:\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Anaconda3\lib\site-packages\keras\engine\sequential.py:111: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


756/756 [==============================] - 7s 9ms/step


In [3]:
counter = 0
for seq_index in range(10):
  print('-')
  print('Original input: ', preX_train[seq_index])
  print('Input sentence:', simple_train_sentences[seq_index])
  print('Decoded sentence:', prey_train[counter:counter+len(simple_train_sentences[seq_index])])
  counter += len(simple_train_sentences[seq_index])

-
Original input:  [('In', 'IN'), ('July', 'NNP'), (',', ','), ('the', 'DT'), ('Environmental', 'NNP'), ('Protection', 'NNP'), ('Agency', 'NNP'), ('imposed', 'VBD'), ('a', 'DT'), ('gradual', 'JJ'), ('ban', 'NN'), ('on', 'IN'), ('virtually', 'RB'), ('all', 'DT'), ('uses', 'NNS'), ('of', 'IN'), ('asbestos', 'NN'), ('.', '.')]
Input sentence: ['In', 'July', ',', 'the', 'Environmental', 'Protection', 'Agency', 'imposed', 'a', 'gradual', 'ban', 'on', 'virtually', 'all', 'uses', 'of', 'asbestos', '.']
Decoded sentence: ['IN', 'NNP', ',', 'DT', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'RB', 'DT', 'NNS', 'IN', 'NN', '.']
-
Original input:  [('By', 'IN'), ('1997', 'CD'), (',', ','), ('almost', 'RB'), ('all', 'DT'), ('remaining', 'VBG'), ('uses', 'NNS'), ('of', 'IN'), ('cancer-causing', 'JJ'), ('asbestos', 'NN'), ('will', 'MD'), ('be', 'VB'), ('outlawed', 'VBN'), ('*-6', '-NONE-'), ('.', '.')]
Input sentence: ['By', '1997', ',', 'almost', 'all', 'remaining', 'uses', 'of', 'cancer-caus

In [4]:
def get_part_of_speech(word):
  probable_part_of_speech = wordnet.synsets(word)
  pos_counts = Counter()
  pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
  pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
  pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
  pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )
  
  most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
  return most_likely_part_of_speech


In [5]:
def clean_up_example(text):
    cleaned = re.sub('\W+', ' ', text)
    tokenized = word_tokenize(cleaned)
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized]
    return lemmatized

#These properties could include informations about
# previous and next words as well as prefixes and suffixes.
def add_basic_features(sentence_terms, index):
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }


def give_tag(probabilities):
    #Take out index number of most probable in each list, then decodes them    
    maximum = probabilities.max()
    index_of_maximum = np.where(probabilities == maximum)
    tag = encoder_model.inverse_transform(index_of_maximum[0])
    return tag

In [7]:
example = clean_up_example("walk downn")
lem_list = []

for i in range(len(example)): 
    lem_list.append(add_basic_features(example, i))

example_result = training_model.predict(dict_vectorizer.transform(lem_list))

for i in range(len(lem_list)):
  print('-')
  print('Input word:', example[i])
  print('Decoded tag:',give_tag(example_result[i]))


-
Input word: walk
Decoded tag: ['NN']
-
Input word: downn
Decoded tag: ['NN']


In [8]:
system("cls")
system("color b9")
FILE = open("data.txt", "r")
listOFCommands = FILE.readlines()
post_data = {}
post_data['command'] = []

for command in listOFCommands:
    command_list = clean_up_example(command)
    temp_list = []
    temp_tags = []
    for i in range(len(command_list)): 
        temp_list.append(add_basic_features(command, i))
    tagged_command = training_model.predict(dict_vectorizer.transform(temp_list))
    for i in range(len(command_list)):
        temp_tags.append(give_tag(tagged_command[i])[0])
    post_data['command'].append({'sentence' : command, 'tags' : temp_tags})
        
data_path = 'data_commands.txt'
with open(data_path, 'w') as outfile:
    json.dump(post_data, outfile)

FILE.close()


In [ ]:
training_model = load_model('cnn.h5')
pkl_file = open('Departure_encoder.pkl', 'rb')
encoder_model = pickle.load(pkl_file)
pkl_file.close()
pkl_file = open('dict_vectorizer.pkl', 'rb')
dict_vectorizer = pickle.load(pkl_file)
pkl_file.close()

def get_part_of_speech(word):
    probable_part_of_speech = wordnet.synsets(word)
    pos_counts = Counter()
    pos_counts["n"] = len([item for item in probable_part_of_speech if item.pos() == "n"])
    pos_counts["v"] = len([item for item in probable_part_of_speech if item.pos() == "v"])
    pos_counts["a"] = len([item for item in probable_part_of_speech if item.pos() == "a"])
    pos_counts["r"] = len([item for item in probable_part_of_speech if item.pos() == "r"])
    most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
    return most_likely_part_of_speech


def clean_up_example(text):
    cleaned = re.sub('\W+', ' ', text)
    tokenized = word_tokenize(cleaned)
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized]
    return lemmatized

#These properties could include informations about
# previous and next words as well as prefixes and suffixes.
def add_basic_features(sentence_terms, index):
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }

def give_tag(probabilities):
    #Take out index number of most probable in each list, then decodes them
    maximum = probabilities.max()
    index_of_maximum = np.where(probabilities == maximum)
    tag = encoder_model.inverse_transform(index_of_maximum[0])
    return tag

example = clean_up_example("walk down")
lem_list = []

for i in range(len(example)):
    lem_list.append(add_basic_features(example, i))

example_result = training_model.predict(dict_vectorizer.transform(lem_list))

for i in range(len(lem_list)):
    print('-')
    print('Input word:', example[i])
    print('Decoded tag:', give_tag(example_result[i]))


In [7]:
training_model = load_model('cnn.h5')
pkl_file = open('Departure_encoder.pkl', 'rb')
encoder_model = pickle.load(pkl_file)
pkl_file.close()
pkl_file = open('dict_vectorizer.pkl', 'rb')
dict_vectorizer = pickle.load(pkl_file)
pkl_file.close()

def get_part_of_speech(word):
    probable_part_of_speech = wordnet.synsets(word)
    pos_counts = Counter()
    pos_counts["n"] = len([item for item in probable_part_of_speech if item.pos() == "n"])
    pos_counts["v"] = len([item for item in probable_part_of_speech if item.pos() == "v"])
    pos_counts["a"] = len([item for item in probable_part_of_speech if item.pos() == "a"])
    pos_counts["r"] = len([item for item in probable_part_of_speech if item.pos() == "r"])
    most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
    return most_likely_part_of_speech


def clean_up_example(text):
    cleaned = re.sub('\W+', ' ', text)
    tokenized = word_tokenize(cleaned)
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized]
    return lemmatized

#These properties could include informations about
# previous and next words as well as prefixes and suffixes.
def add_basic_features(sentence_terms, index):
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }

def give_tag(probabilities):
    #Take out index number of most probable in each list, then decodes them
    maximum = probabilities.max()
    index_of_maximum = np.where(probabilities == maximum)
    tag = encoder_model.inverse_transform(index_of_maximum[0])
    return tag

example = clean_up_example("walk down")
lem_list = []

for i in range(len(example)):
    lem_list.append(add_basic_features(example, i))

example_result = training_model.predict(dict_vectorizer.transform(lem_list))

for i in range(len(lem_list)):
    print('-')
    print('Input word:', example[i])
    print('Decoded tag:', give_tag(example_result[i]))


D:\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


-
Input word: walk
Decoded tag: ['NN']
-
Input word: down
Decoded tag: ['NN']
